In [ ]:
import itertools
import networkx as nx
import matplotlib.pyplot as plt
# Create a directed graph
G = nx.DiGraph()

# Add nodes for all combinations
for combination in ListExperiences_:
    G.add_node(tuple(combination))

# Add edges for hierarchical dependency
for i, combination in enumerate(ListExperiences_):
    for j in range(i + 1, len(ListExperiences_)):
        if set(ListExperiences_[i]).issubset(ListExperiences_[j]):
            G.add_edge(tuple(ListExperiences_[i]), tuple(ListExperiences_[j]))

# Draw the graph
pos = nx.spring_layout(G)  # Position nodes using a spring layout algorithm
nx.draw(G, pos, with_labels=True, node_size=50, node_color='lightblue', font_size=8, font_weight='bold')
plt.title('Hierarchical Dependency between Combinations')
plt.show()

In [ ]:
df=pd.read_csv("zoo.csv")
df

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GraphConvolutionLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphConvolutionLayer, self).__init__()
        self.weight = nn.Parameter(torch.Tensor(in_features, out_features))
        self.bias = nn.Parameter(torch.Tensor(out_features))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)
        nn.init.zeros_(self.bias)

    def forward(self, adjacency_matrix, node_features):
        support = torch.matmul(node_features, self.weight)
        output = torch.matmul(adjacency_matrix, support) + self.bias
        return output

class GCN(nn.Module):
    def __init__(self, input_features, hidden_features, output_classes):
        super(GCN, self).__init__()
        self.layer1 = GraphConvolutionLayer(input_features, hidden_features)
        self.layer2 = GraphConvolutionLayer(hidden_features, output_classes)

    def forward(self, adjacency_matrix, node_features):
        h1 = F.relu(self.layer1(adjacency_matrix, node_features))
        output = self.layer2(adjacency_matrix, h1)
        return F.log_softmax(output, dim=1)  # Using log_softmax for classification



In [ ]:
L2_=[k for k in ListExperiences_ if k not in ListExperiences] 
print("Missing elements:", L2_)

In [ ]:
exper=os.listdir("checkpoints/")
missing=0
L_missing=[]
L_missingL=[]
for folder in exper:
    if len(os.listdir("checkpoints/{}".format(folder)))<3 and ("gelu" not in os.listdir("checkpoints/{}".format(folder))) :
        missing=missing+1
        L_missing.append(folder)
for folder in L_missing :       
    L_class=[]
    for classe in folder[1:-1].split(","):
        L_class.append(int(classe))
    L_missingL.append(L_class)           
len(L_missingL)

In [ ]:
from torch.utils.data import TensorDataset, ConcatDataset, DataLoader
L_Datasets=[]
L_activations=["gelu","relu"]#,"silu","leakyrelu","sigmoid","tanh"]
for activ in L_activations: 
    dsa=torch.load(f'train_dataset_{activ}.pt')
    L_Datasets.append(dsa)
train_dsab_cat = ConcatDataset(L_Datasets)
train_loader = DataLoader(train_dsab_cat,batch_size=20)


In [ ]:
from sklearn.manifold import MDS
import pandas as pd

# Define the number of components for MDS
n_components = 48

# Apply MDS for dimensionality reduction
mds_tr = MDS(n_components=n_components)
train_reduced_data = mds_tr.fit_transform(train_labels)
test_reduced_data = mds_ts.fit_transform(test_labels)

train_reduced_data.shape,test_reduced_data.shape

In [ ]:
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from ripser import ripser
from persim import plot_diagrams
from sklearn.model_selection import train_test_split

# Generate sample data
np.random.seed(42)
data = train_features  # Example 1D vectors of length 2464

# Standardize the data
scaler = StandardScaler()
data = scaler.fit_transform(data)

# Compute persistent homology using scikit-tda
diagrams = ripser(data, maxdim=48)['dgms']

diagrams[0][-1]=diagrams[0][-2]
#print(diagrams)
# Visualize the persistent homology diagrams
plot_diagrams(diagrams, show=True)

# Extract features from the persistent homology diagram
features = np.array([np.mean(diagram[:, 1] - diagram[:, 0]) for diagram in diagrams])

# Reshape the features array to make it 2D
features = features.reshape(-1, 1)

# Normalize the features (optional)
features = (features - np.mean(features)) / np.std(features)

# Convert features to PyTorch tensor
features_tensor = torch.FloatTensor(features)

# Instantiate the autoencoder
input_size = features.shape[1]
encoding_size = 48  # Choose an appropriate encoding size
autoencoder = Autoencoder(input_size, encoding_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

# Training loop
num_epochs = 1000

for epoch in range(num_epochs):
    # Forward pass
    reconstructed_data = autoencoder(features_tensor)

    # Compute the loss
    loss = criterion(reconstructed_data, features_tensor)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print or log the loss for monitoring training progress
    if epoch % 100 == 0:
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

# Visualize the results
with torch.no_grad():
    encoded_data = autoencoder.encoder(features_tensor).numpy()


In [ ]:
import networkx as nx

  # Assuming the first 10 columns are labels
 # Assuming the rest are features

# Create an adjacency matrix based on the labels
train_adjacency_matrix = np.dot(train_features, train_features.T)

train_graph = nx.Graph(train_adjacency_matrix)

# Add node features to the graph
for node_index, feature_vector in enumerate(train_features):
    train_graph.nodes[node_index]['features'] = train_reduced_data
    
    
test_adjacency_matrix = np.dot(test_features, test_features.T)

test_graph = nx.Graph(test_adjacency_matrix)

# Add node features to the graph
for node_index, feature_vector in enumerate(test_features):
    test_graph.nodes[node_index]['features'] = test_reduced_data

In [ ]:
node_size = 20 # Set the size of nodes
label_font = {'fontweight': 'bold', 'fontsize': 8}  # Set label font properties

# Visualize the graph
pos = nx.spring_layout(graph)
fig, ax = plt.subplots(figsize=(80, 60))  # Set the figure size
nx.draw(graph, pos, with_labels=False, node_size=node_size, ax=ax)
nx.draw_networkx_labels(graph, pos, font_size=8, font_family="sans-serif", font_color="red",alpha=0.2)

plt.show()

In [ ]:
#!pip install torch_geometric
import torch
from torch_geometric.data import Data

# Convert NetworkX graph to PyTorch Geometric Data
edges = np.array(train_graph.edges()).T
train_edge_index = torch.tensor(edges, dtype=torch.long)
x_train = torch.tensor(train_features, dtype=torch.float)
y_train = torch.tensor(train_reduced_data, dtype=torch.float)

edges = np.array(test_graph.edges()).T
test_edge_index = torch.tensor(edges, dtype=torch.long)
x_test = torch.tensor(test_features, dtype=torch.float)
y_test = torch.tensor(test_reduced_data, dtype=torch.float)
train_data = Data(x=x_train, edge_index=train_edge_index, y=y_train)
test_data = Data(x=x_test, edge_index=test_edge_index, y=y_test)

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader

class GCN(nn.Module):
    def __init__(self, input_features, hidden_features, output_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_features, hidden_features)
        self.conv2 = GCNConv(hidden_features, output_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

# Initialize model, loss, optimizer
model = GCN(input_features=train_features.shape[1], hidden_features=64, output_classes=train_reduced_data.shape[1])
criterion = nn.L1Loss()
optimizer = Adam(model.parameters(), lr=0.05)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-3, max_lr=0.1, step_size_up=200, mode="triangular2", cycle_momentum=False)

data_list = [train_data]
loader = DataLoader(data_list, batch_size=8)
L=[]
L_tr_mape=[]
L_ts_mape=[]
# Train the model
num_epochs = 5000
for epoch in range(num_epochs):
    for batch in loader:
            model.train()
            optimizer.zero_grad()
            output = model(batch)
            loss = criterion(output, batch.y)
            loss.backward()
            optimizer.step()
            L.append(loss.item())
    scheduler.step()
    if epoch%30==0:
        mape_train=calculate_mape(train_data.y.detach().numpy(), output.detach().numpy())
        L_tr_mape.append(mape_train)
        preds=model(test_data)
        mape_test_result = calculate_mape(test_data.y.detach().numpy(), preds.detach().numpy())
        L_ts_mape.append(mape_test_result)
        print( f"epoch {epoch} || 30 epoch mean MAE :{ (sum(L[-30:]) / len(L[-30:])):.2f} || last MAE {L[-1]} || train MAPE {mape_train}% || test MAPE {mape_test_result}%"  )

# You can now use the trained model for predictions